In [1]:
import polars as pl
import pandas as pd
import pyspark as ps

In [2]:
CSV_40M_PATH = 'data/Parking_Violations_40M.csv' # 9.65 GB
CSV_20M_PATH = 'data/Parking_Violations_20M.csv' # 4.89 GB
CSV_9M_PATH = 'data/Parking_Violations_9M.csv' # 1.95 GB
CSV_4M_PATH = 'data/Parking_Violations_4M.csv' # 855 MB
CSV_2M_PATH = 'data/Parking_Violations_2M.csv' # 428 MB
CSV_1M_PATH = 'data/Parking_Violations_1M.csv' # 214 MB
CSV_500K_PATH = 'data/Parking_Violations_500K.csv' # 107 MB

In [3]:
CSV_PATH = CSV_500K_PATH
OUT_JSON_NAME = 'CSV_500K_PATH'

In [4]:
comparison_results_dict = {
    'read_csv': {},
    'filter_one_condition': {},
    'filter_multiple_conditions': {},
    'create_new_column': {},
    'group_by': {}
}

# Let's compare

## 1. Read CSV

### 1.1. Polars

In [5]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
polars_df.collect()

113 ms ± 2.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 113 ms ± 2.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [6]:
comparison_results_dict['read_csv']['polars'] = _.best

### 1.2. Pandas

In [7]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')

783 ms ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 783 ms ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [8]:
comparison_results_dict['read_csv']['pandas'] = _.best

## 2. Filter rows based on one condition

### 2.1. Polars

In [9]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.filter(pl.col(['Registration State']) == "NY")
result_polars_df.collect()

122 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 122 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [10]:
comparison_results_dict['filter_one_condition']['polars'] = _.best

### 2.2. Pandas

In [11]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
results_pandas_df = pandas_df.loc[pandas_df['Registration State'] == 'NY']

849 ms ± 53.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 849 ms ± 53.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [12]:
comparison_results_dict['filter_one_condition']['pandas'] = _.best

## 3. Filter rows based on multiple conditions

### 3.1. Polars

In [13]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.filter(
    (pl.col('Registration State') == 'NY') & \
    (pl.col('Summons Number') <= 1335096139) & \
    (pl.col('Summons Number') > 1335089433)
)
result_polars_df.collect()

123 ms ± 8.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 123 ms ± 8.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [14]:
comparison_results_dict['filter_multiple_conditions']['polars'] = _.best

### 3.2. Pandas

In [15]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
results_pandas_df = pandas_df.loc[
    (pandas_df['Registration State'] == 'NY') & \
    (pandas_df['Summons Number'] <= 1335096139) & \
    (pandas_df['Summons Number'] > 1335089433)
]

804 ms ± 9.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 804 ms ± 9.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [16]:
comparison_results_dict['filter_multiple_conditions']['pandas'] = _.best

## 4. Create new column based on another

### 4.1. Polars

In [17]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.with_columns([
    (pl.col("Violation Code") + 10_000).alias('Clean Violation Code')
])
result_polars_df.collect()

116 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 116 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [18]:
comparison_results_dict['create_new_column']['polars'] = _.best

### 4.2. Pandas

In [19]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
pandas_df['Clean Violation Code'] = pandas_df['Violation Code'].apply(lambda x: x + 10_000)

900 ms ± 79.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 900 ms ± 79.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [20]:
comparison_results_dict['create_new_column']['pandas'] = _.best

## 5. GroupBy

### 5.1. Polars

In [21]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.group_by(by='Registration State').count()
result_polars_df.collect()

<magic-timeit>:2: DeprecationWarning: `count` was renamed; use `len` instead


28.4 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 28.4 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [22]:
comparison_results_dict['group_by']['polars'] = _.best

### 5.2. Pandas

In [23]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
grouped_pandas_df = pandas_df.groupby(by='Registration State').count()

1.04 s ± 47.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1.04 s ± 47.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [24]:
comparison_results_dict['group_by']['pandas'] = _.best

## 6. RESULTS

In [25]:
import json
print(json.dumps(comparison_results_dict, indent=4))

{
    "read_csv": {
        "polars": 0.10926566700800322,
        "pandas": 0.7538160830008565
    },
    "filter_one_condition": {
        "polars": 0.11966282920038793,
        "pandas": 0.8192257090122439
    },
    "filter_multiple_conditions": {
        "polars": 0.11417679590085754,
        "pandas": 0.7933447500108741
    },
    "create_new_column": {
        "polars": 0.11217769170034445,
        "pandas": 0.8473366250109393
    },
    "group_by": {
        "polars": 0.026752233399020043,
        "pandas": 1.002640375008923
    }
}


In [26]:
with open(f'{OUT_JSON_NAME}_with_read.json', 'w') as file:
    json.dump(comparison_results_dict, file, indent=4)